In [1]:
import cv2
import mediapipe as mp
import socket
import numpy as np
import math
import json

# MediaPipe Hands 모듈 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# 웹캠 설정
cap = cv2.VideoCapture(0)

# TCP 서버 설정
def send_hand_landmarks(host='127.0.0.1', port=6521):
    # 소켓 객체 생성
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as client_socket:
        # 서버에 연결
        client_socket.connect((host, port))
        print(f"서버에 연결되었습니다. {host}:{port}")

        while True:
            # 웹캠에서 프레임 읽기
            success, image = cap.read()
            if not success:
                continue

            # 이미지를 RGB로 변환
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # 성능을 위해 이미지 쓰기 불가능으로 설정
            image.flags.writeable = False

            # MediaPipe를 통해 손 검출
            results = hands.process(image)

            # 이미지를 RGB에서 BGR로 변환하여 그리기 작업 준비
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # 검출된 손의 랜드마크를 이미지에 그림
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                        mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2))
                    
                    # 손목, 검지, 엄지 좌표 가져오기
                    wrist = np.array([hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x,
                                      hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y,
                                      hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].z])
                    index = np.array([hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x,
                                      hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y,
                                      hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].z])
                    thumb = np.array([hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x,
                                      hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y,
                                      hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].z])
                    
                    # 여기서 손 모양을 분류하고, 분류 결과를 전송
                    hand_shape = classify_hand_shape(hand_landmarks)
                    # 예를 들어, 가위는 0, 바위는 1, 보는 2로 가정
                    
                    # 랜드마크들의 좌표 값을 소수점 셋째 자리까지 반올림하여 문자열로 변환하여 서버로 전송
                    landmarks_str = f"{round(thumb[0], 3)},{round(thumb[1], 3)},{round(thumb[2], 3)};" \
                                    f"{round(index[0], 3)},{round(index[1], 3)},{round(index[2], 3)};" \
                                    f"{round(wrist[0], 3)},{round(wrist[1], 3)},{round(wrist[2], 3)};" \
                                    f"{hand_shape}/"
                    print(landmarks_str)
                    client_socket.sendall(landmarks_str.encode())
                    
                    # 데이터를 JSON 형식으로 묶어서 전송
                    # data = {
                    #     "thumb": {
                    #         "x": round(thumb[0], 3),
                    #         "y": round(thumb[1], 3),
                    #         "z": round(thumb[2], 3)
                    #     },
                    #     "index": {
                    #         "x": round(index[0], 3),
                    #         "y": round(index[1], 3),
                    #         "z": round(index[2], 3)
                    #     },
                    #     "wrist": {
                    #         "x": round(wrist[0], 3),
                    #         "y": round(wrist[1], 3),
                    #         "z": round(wrist[2], 3)
                    #     },
                    #     "hand_shape": hand_shape
                    # }
                    # json_data = json.dumps(data)
                    # print(json_data)
                    # client_socket.sendall(json_data.encode())


                    

            # 결과 이미지 표시
            cv2.imshow('Hand Tracking', image)

            # 'q'를 눌러 종료
            if cv2.waitKey(5) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

# 여기에 손 모양을 분류하는 함수를 정의합니다.
def classify_hand_shape(hand_landmarks):
     # 손가락 끝과 PIP(손가락 중간 관절) 사이의 거리를 비교하여 손가락이 펴져 있는지 확인
    open_fingers = []
    for i in [4, 8, 12, 16, 20]:  # 엄지부터 새끼손가락까지의 끝 랜드마크 인덱스
        tip = hand_landmarks.landmark[i]  # 손가락 끝
        pip = hand_landmarks.landmark[i - 2]  # 손가락 중간 관절

        #if handedness.classification[0].label == "Right":  # 오른손인 경우 판별하기
        #    if i == 4:  # 엄지손가락인 경우
        #        open_fingers.append(tip.x < pip.x)
                
        # 엄지는 x좌표를, 나머지 손가락은 y좌표를 사용하여 개폐 상태 확인 - 오른손 기준
        if i == 4:  # 엄지손가락인 경우
            open_fingers.append(tip.x > pip.x)
        else:
            open_fingers.append(tip.y < pip.y)
    
    # '가위', '바위', '보' 판별
    if open_fingers.count(True) == 5:
        return 2
    elif open_fingers.count(True) == 2 and open_fingers[1]:
        return 1
    elif open_fingers.count(True) == 0:
        return 0
    else:
        return 3

if __name__ == "__main__":
    send_hand_landmarks()

서버에 연결되었습니다. 127.0.0.1:6521
0.287,0.849,-0.053;0.195,0.776,-0.054;0.301,1.076,0.0;3/
0.277,0.826,0.014;0.192,0.757,0.012;0.315,1.048,0.0;3/
0.265,0.715,0.092;0.193,0.668,0.114;0.323,0.947,0.0;3/
0.271,0.703,0.077;0.196,0.648,0.103;0.311,0.911,0.0;3/
0.272,0.685,0.107;0.201,0.643,0.148;0.307,0.89,0.0;3/
0.292,0.657,0.144;0.223,0.615,0.214;0.31,0.833,0.0;3/
0.296,0.651,0.149;0.23,0.608,0.223;0.314,0.809,0.0;3/
0.3,0.641,0.163;0.236,0.602,0.228;0.317,0.792,0.0;3/
0.306,0.637,0.161;0.238,0.597,0.225;0.316,0.776,0.0;3/
0.309,0.632,0.178;0.239,0.596,0.254;0.316,0.767,0.0;3/
0.307,0.618,0.182;0.23,0.586,0.274;0.314,0.741,0.0;3/
0.305,0.62,0.175;0.23,0.586,0.266;0.319,0.736,0.0;3/
0.306,0.62,0.172;0.229,0.591,0.257;0.321,0.732,0.0;3/
0.331,0.651,0.14;0.268,0.597,0.176;0.329,0.787,0.0;3/
0.356,0.674,0.136;0.3,0.598,0.187;0.342,0.834,0.0;3/
0.391,0.717,0.014;0.344,0.605,0.029;0.358,0.898,0.0;3/
0.332,0.802,-0.023;0.243,0.678,-0.034;0.296,1.075,0.0;3/
0.304,0.724,0.01;0.219,0.619,0.018;0.309,1.01